In [ ]:
src_path = r"vector\\"
target_path = "output\\"
trim_files(src_path,target_path)

In [ ]:
# 查看各种语句的几个例子

object_func_call  = set()
folder_path = '.\output\\'
file_list = read_dir(folder_path)
for filename in file_list:
    with open(folder_path + filename, 'r') as f:
        for line in f:
            if is_object_func_call(line):
                res = parse_object_func_call(line)
                object_func_call.add(res[1])
                
print(object_func_call)

In [4]:
from tool_function import *
from graph import *

# 使用解析好的资源构建链表或者图
# 首先是设计图的节点类的结构，然后是边的关系，最后是图的结构
# 读取一个文件，解析每一行，判断每一行语句的种类，然后进行处理
# 1. 读取文件
# 2. 解析每一行
# 3. 判断语句种类
# 4. 处理语句
graph = Graph()
filename = r"output\3DMarkICFWorkload_58.sdx";
line_pos = 0

with open(filename, "r") as f:
    for line in f:
        if is_comment(line):
            pass
        elif is_empty(line):
            pass
        elif is_other(line):
            res = parse_statement(line)
            node = Node(res[1],[filename,line_pos])
            graph.add_vertex(node)
        elif is_object_func_call(line):
            res = parse_object_func_call(line) # 解析对象调用
            print(res[2])
            if res[1] in create_func:
                # 如果在创建函数中，会根据已有的资源创建一个对象，然后连接起他们
                node = Node(res[1],[filename,line_pos])
                graph.add_vertex(node)
            elif res[1] in set_func:
                pass
                # 如果在设置函数，将已有的一个资源设置为另一个资源的属性
            elif res[1] in other_func:
                # 如果在其他的函数中，不做任何事情
                pass
        elif is_func_call(line):
            pass
            #print(line_pos,parse_func_call(line))
        
        line_pos = line_pos+1
# graph.print_vertices()
# graph.print()


['Buffer = 0', 'riid = IID_ID3D11Texture2D', 'ppSurface = pSwap_0_buf_0']
['pRenderTargetView = pRTV_1', 'ColorRGBA = color_194']
['pResourceView = pRTV_22']
['pResource = pBuf_13', 'Subresource = 0x0', 'MapType = D3D11_MAP_WRITE_DISCARD', 'MapFlags = 0x00000000', 'pMappedResource = pMapRes_9860']
['pResource = pBuf_13', 'Subresource = 0']
['pResource = pBuf_15', 'Subresource = 0x0', 'MapType = D3D11_MAP_WRITE_DISCARD', 'MapFlags = 0x00000000', 'pMappedResource = pMapRes_9861']
['pResource = pBuf_15', 'Subresource = 0']
['pResource = pBuf_16', 'Subresource = 0x0', 'MapType = D3D11_MAP_WRITE_DISCARD', 'MapFlags = 0x00000000', 'pMappedResource = pMapRes_9862']
['pResource = pBuf_16', 'Subresource = 0']
['pResource = pBuf_17', 'Subresource = 0x0', 'MapType = D3D11_MAP_WRITE_DISCARD', 'MapFlags = 0x00000000', 'pMappedResource = pMapRes_9863']
['pResource = pBuf_17', 'Subresource = 0']
['pResourceView = pRTV_9']
['pResourceView = pRTV_10']
['pResource = pTex2D_99']
['pRenderTargetView = pRT